In [ ]:
import json
import os
from glob import glob
from collections import defaultdict

import numpy as np
import pandas as pd
from tqdm.notebook import tqdm

In [ ]:
import torch

train_on_gpu = torch.cuda.is_available()

if not train_on_gpu:
    print('CUDA is not available.  Training on CPU ...')
else:
    print('CUDA is available!  Training on GPU ...')

CUDA is available!  Training on GPU ...


In [ ]:
from google.colab import drive
drive.mount('/content/myDrive')

Drive already mounted at /content/myDrive; to attempt to forcibly remount, call drive.mount("/content/myDrive", force_remount=True).


In [ ]:
# os.chdir('myDrive/MyDrive/ufru')

In [ ]:
# !rm -rf public_data
!rm -rf train_data

In [ ]:
# !unzip public_data.zip > public_data_std_out
!unzip train_data.zip > train_data_std_out
# !rm public_data_std_out
!rm train_data_std_out

### Read y_train

In [ ]:
y = pd.read_csv('train_anwers.csv', sep='\t').astype(int)
y_train = y.target
y.head()

,name,target
0,29970,1
1,18654,0
2,29133,0
3,20927,0
4,5526,0


### Read train data as jsons

In [ ]:
raw_data = {}
for name in tqdm(y['name']):
    with open(f'train_data/{name}.json', 'r', encoding='utf-8') as inp:
        raw_data[name] = json.load(inp)

  0%|          | 0/8000 [00:00<?, ?it/s]

### Remove nesting in dict and get keys

In [ ]:
flatten_keys = defaultdict(int)
def get_flatten_keys(d, parent_key='', sep='.'):
    if d is None:
        return
    next_sep = sep
    sep = sep if parent_key else ''
    if isinstance(d, dict):
        for k, v in d.items():
            get_flatten_keys(v, f"{parent_key}{sep}{k}", sep=next_sep)
    elif isinstance(d, list):
        for i, item in enumerate(d):
            get_flatten_keys(item, f"{parent_key}{sep}{i}", sep=next_sep)
    else:
        flatten_keys[parent_key] += 1

In [ ]:
for data in tqdm(raw_data.values()):
    get_flatten_keys(data)
flatten_keys = dict(flatten_keys)

  0%|          | 0/8000 [00:00<?, ?it/s]

In [ ]:
len(flatten_keys)

93270

### Delete keys where NaN more than 70%

In [ ]:
useless_keys = set()
ln = 8000
for key, val in flatten_keys.items():
    if val < ln * 0.1:
        useless_keys.add(key)

In [ ]:
len(useless_keys)

92283

In [ ]:
for key in useless_keys:
    del flatten_keys[key]

In [ ]:
len(flatten_keys)

987

### Get data by keys

In [ ]:
def extract_features(data, features_names):
    def get_value(obj, keys_list):
        if not keys_list:
            return obj
        key = keys_list.pop(0)
        key = int(key) if key.isdigit() else key
        try:
            return get_value(obj[key], keys_list)
        except:
            return None
        
    train_data = defaultdict(list)

    for i, game_data in tqdm(enumerate(data.values())):
        for key_str in features_names:
            keys = key_str.split('.')
            val = get_value(game_data, keys)
                    
            train_data[key_str].append(val)
    print(train_data)
    df = pd.DataFrame(train_data)
    return df
        

In [ ]:
def preprocess_features(data):
    df = data.copy()
    # replace nans with median value
    for col in tqdm(df.columns):
        if df[col].isna().sum() > 0:
            if df[col].dtype in ['int64', 'float64', 'bool']:
                df[col].fillna(df[col].median(), inplace=True)
            else:
                ind = df[col].value_counts().index
                if len(ind) != 0:
                    df[col].fillna(ind[0], inplace=True)
                else:
                    df[col].fillna(0)

    # bool features to int
    for col in df.select_dtypes(include=['bool']).columns.tolist():
        df[col] = df[col].astype(int)

    return df

In [ ]:
X_train = extract_features(raw_data, flatten_keys)

0it [00:00, ?it/s]

IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)



In [ ]:
X_train2 = preprocess_features(X_train)

  0%|          | 0/987 [00:00<?, ?it/s]

In [ ]:
X_train2

,cluster,dire_score,duration,engine,first_blood_time,game_mode,human_players,leagueid,lobby_type,match_seq_num,...,players.3.ability_upgrades_arr.20,players.3.ability_upgrades_arr.21,players.5.ability_upgrades_arr.20,players.5.ability_upgrades_arr.21,players.3.permanent_buffs.1.permanent_buff,players.3.permanent_buffs.1.stack_count,players.3.permanent_buffs.1.grant_time,players.3.personaname,players.3.rank_tier,players.9.ability_upgrades_arr.21
0,183,34,2022,1,100,0,10,0,0,5780373336,...,5045.5,2432.0,5370.0,2166.0,12.0,0.0,1263.0,.,31.0,1069.0
1,-123,47,2648,0,54,22,10,0,7,0,...,5045.5,2432.0,5370.0,2166.0,12.0,0.0,1263.0,.,31.0,1069.0
2,273,65,2684,1,101,18,10,0,0,5780373039,...,5045.5,2432.0,5370.0,2166.0,12.0,0.0,1263.0,.,31.0,1069.0
3,184,83,0,0,20,22,-1,0,7,5780347103,...,5045.5,2432.0,5370.0,2166.0,12.0,0.0,1263.0,.,31.0,1069.0
4,154,65,0,1,101,22,10,0,7,5780418598,...,5045.5,2432.0,5370.0,2166.0,12.0,0.0,1263.0,.,31.0,1069.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7995,-40,56,2531,1,110,22,0,0,7,5780379689,...,5045.5,2432.0,5370.0,2166.0,12.0,0.0,1650.0,.,31.0,1069.0
7996,184,33,1158,1,158,23,10,0,0,5780312140,...,5045.5,2432.0,5370.0,2166.0,12.0,0.0,1263.0,.,31.0,1069.0
7997,274,40,88,1,104,22,0,0,7,5780365863,...,5045.5,2432.0,5370.0,2166.0,12.0,0.0,1263.0,.,31.0,1069.0
7998,153,20,2200,0,76,7,10,0,7,5780378245,...,5045.5,2432.0,5370.0,2166.0,12.0,0.0,1263.0,Perfect Blue,0.0,1069.0


### Remove features that correlates with target less that 0.05

In [ ]:
len(y_train)

8000

In [ ]:
df_corr = X_train2.copy()
df_corr['target'] = y_train
corrs = df_corr.corr()['target'].abs() > 0.1
to_drop = []
for col in corrs.index:
    if not corrs[col]:
        if col != 'target':
            to_drop.append(col)

In [ ]:
X_train3 = X_train2.drop(columns=to_drop)

In [ ]:
final_features = X_train3.columns

In [ ]:
X_train3

,dire_score,radiant_score,players.0.assists,players.0.deaths,players.0.gold,players.0.tower_damage,players.0.kills_per_min,players.0.kda,players.0.benchmarks.kills_per_min.raw,players.0.benchmarks.kills_per_min.pct,...,players.8.benchmarks.kills_per_min.raw,players.8.benchmarks.kills_per_min.pct,players.9.personaname,players.9.benchmarks.tower_damage.raw,players.9.benchmarks.tower_damage.pct,players.7.personaname,players.7.benchmarks.kills_per_min.raw,players.7.benchmarks.kills_per_min.pct,players.4.personaname,players.3.personaname
0,34,63,8.0,7.0,661.0,8797.0,-0.016677,1.0,0.148368,0.125000,...,0.161182,0.500000,.,510.0,0.420345,.,0.158103,0.500000,< blank >,.
1,47,40,6.0,6.0,4116.0,0.0,0.294562,2.0,0.294562,-0.212283,...,0.249245,0.928571,Churka,3023.0,0.826087,.,0.158103,0.500000,< blank >,.
2,65,0,10.0,6.0,1252.0,618.0,0.169891,1.0,0.163732,0.519615,...,0.161182,0.500000,.,510.0,0.420345,.,0.158103,0.500000,< blank >,.
3,83,40,8.0,14.0,390.0,1079.0,0.164062,1.0,0.164062,0.400000,...,0.375000,1.000000,qwe,8046.0,1.000000,Спуди Мун,0.773438,1.000000,< blank >,.
4,65,85,-11.0,12.0,0.0,665.0,0.123267,3.0,0.123267,0.550000,...,0.030817,0.125000,Tinie Winnie Bittie,5916.0,-0.117160,.,0.323575,0.800000,< blank >,.
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7995,56,39,21.0,0.0,342.0,0.0,0.118530,1.0,0.118530,-0.137301,...,0.284473,0.900000,.,5618.0,0.833333,.,0.426709,0.030604,< blank >,.
7996,33,18,10.0,6.0,624.0,0.0,0.051813,1.0,0.051813,0.181818,...,0.000000,0.100000,No Face Bando,2952.0,0.795455,.,0.051813,0.174419,< blank >,.
7997,40,54,17.0,12.0,303.0,2040.0,0.167464,1.0,0.167464,1.000000,...,0.263158,0.696970,.,0.0,-0.037863,Hƴdαяηeṧ,0.158103,0.500000,< blank >,.
7998,20,41,15.0,4.0,281.0,557.0,0.163636,4.0,0.163636,0.428571,...,0.136364,0.400000,.,167.0,0.096774,吃藕丑,-0.007182,0.420690,< blank >,Perfect Blue


## Data augmentation

In [ ]:
x_as_dict = X_train3.to_dict(orient='index')

In [ ]:
y_aug = np.zeros(16000)
for i, x in enumerate(y_train):
    y_aug[2 * i] = x
    y_aug[2 * i + 1] = 1 - x
y_aug.shape

(16000,)

In [ ]:
from sklearn.utils import shuffle

In [ ]:
def make_augmented_dict(x_as_dict):
    sep = '.'
    aug_x = {}

    for k in tqdm(x_as_dict.keys()):
        j, z = np.random.randint(5, 10, size=2) # будем свапать игроков из ондной группы в reversed_game
        reversed_game = {} # признаки players
        other = {} # другие
        for key in x_as_dict[k].keys():
            parts_of_key = key.split(sep)
            if parts_of_key[0] == 'players':
                i = int(parts_of_key[1])
                w = i
                if i == j:
                    w = z
                elif i == z:
                    w = j
                reversed_key = parts_of_key[0] + sep + str(9 - w) + sep + sep.join(parts_of_key[2:])
                if reversed_key not in x_as_dict[k]:
                    reversed_game[key] = None
                else:
                    reversed_game[key] = x_as_dict[k][reversed_key]
            else:
                reversed_game[key] = x_as_dict[k][key]
        reversed_game['radiant_score'] = x_as_dict[k]['dire_score']
        reversed_game['dire_score'] = x_as_dict[k]['radiant_score']
        aug_x[f'{k}-0'] = x_as_dict[k]
        aug_x[f'{k}-1'] = reversed_game
    return aug_x

In [ ]:
aug_x = make_augmented_dict(x_as_dict)
X_standart = pd.DataFrame.from_dict(aug_x, orient='index')

  0%|          | 0/8000 [00:00<?, ?it/s]

In [ ]:
cat_cols = [col for col in X_standart.columns if X_standart[col].dtype not in ['int64', 'float64']]
X_standart.drop(columns=cat_cols, inplace=True)
X_standart = preprocess_features(X_standart)

  0%|          | 0/122 [00:00<?, ?it/s]

In [ ]:
X_standart.reset_index(drop=True, inplace=True)

In [ ]:
indicies = shuffle(np.arange(16000))
X_train = X_standart.loc[indicies]
y_train = y_aug[indicies]

# Training

In [ ]:
!pip install catboost

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.ensemble import RandomForestClassifier
from catboost import CatBoostClassifier
import lightgbm as lgb
import warnings
warnings.filterwarnings("ignore")

In [ ]:
rf_param_grid = {
    "n_estimators": [100, 200, 500],
    "max_depth": [None, 5, 10],
    "min_samples_split": [2, 5, 10],
    "min_samples_leaf": [1, 2, 4],
}

cat_param_grid = {
    "iterations": [100, 200, 500],
    "learning_rate": [0.01, 0.05, 0.1],
    "depth": [4, 6, 8],
}

lgb_param_grid = {
    "n_estimators": [100, 200, 500],
    "learning_rate": [0.01, 0.05, 0.1],
    "max_depth": [-1, 5, 10],
}

In [ ]:
big_X_train = X_train
big_y_train = y_train

In [ ]:
X_train = big_X_train[:500]
y_train = big_y_train[:500]

In [ ]:
rf_grid = GridSearchCV(RandomForestClassifier(random_state=42), rf_param_grid, cv=5, n_jobs=-1, verbose=10)

In [ ]:
rf_grid.fit(X_train, y_train)

Fitting 5 folds for each of 81 candidates, totalling 405 fits


GridSearchCV(cv=5, estimator=RandomForestClassifier(random_state=42), n_jobs=-1,
             param_grid={'max_depth': [None, 5, 10],
                         'min_samples_leaf': [1, 2, 4],
                         'min_samples_split': [2, 5, 10],
                         'n_estimators': [100, 200, 500]},
             verbose=10)

In [ ]:
catboost = CatBoostClassifier(random_state=42, silent=True, task_type="GPU")

In [ ]:
result = catboost.grid_search(
    cat_param_grid,
    X_train, y_train,
    cv=5,
    partition_random_seed=42,
    refit=False,
    shuffle=False
)

bestTest = 0.3363886642
bestIteration = 99
0:	loss: 0.3363887	best: 0.3363887 (0)	total: 2.23s	remaining: 57.9s
bestTest = 0.1982685471
bestIteration = 99
1:	loss: 0.1982685	best: 0.1982685 (1)	total: 4.25s	remaining: 53.1s
bestTest = 0.1711029053
bestIteration = 88
2:	loss: 0.1711029	best: 0.1711029 (2)	total: 6.11s	remaining: 48.9s
bestTest = 0.2499860191
bestIteration = 199
3:	loss: 0.2499860	best: 0.1711029 (2)	total: 10.9s	remaining: 1m 2s
bestTest = 0.182606945
bestIteration = 155
4:	loss: 0.1826069	best: 0.1711029 (2)	total: 22.1s	remaining: 1m 37s
bestTest = 0.1679199219
bestIteration = 198
5:	loss: 0.1679199	best: 0.1679199 (5)	total: 33.4s	remaining: 1m 56s
bestTest = 0.1900114441
bestIteration = 497
6:	loss: 0.1900114	best: 0.1679199 (5)	total: 44.6s	remaining: 2m 7s
bestTest = 0.1802310753
bestIteration = 323
7:	loss: 0.1802311	best: 0.1679199 (5)	total: 56.2s	remaining: 2m 13s
bestTest = 0.1672010803
bestIteration = 241
8:	loss: 0.1672011	best: 0.1672011 (8)	total: 1m 10s	

In [ ]:
result['params']

{'depth': 6, 'iterations': 500, 'learning_rate': 0.05}

In [ ]:
catboost = CatBoostClassifier(
    random_seed=42,
    iterations=result['params']['iterations'],
    learning_rate=result['params']['learning_rate'],
    depth=result['params']['depth']
)

In [ ]:
lgb_grid = GridSearchCV(lgb.LGBMClassifier(random_state=42, n_jobs=-1), lgb_param_grid, cv=5, n_jobs=-1, verbose=10)


In [ ]:
lgb_grid.fit(X_train, y_train)

Fitting 5 folds for each of 27 candidates, totalling 135 fits


GridSearchCV(cv=5, estimator=LGBMClassifier(random_state=42), n_jobs=-1,
             param_grid={'learning_rate': [0.01, 0.05, 0.1],
                         'max_depth': [-1, 5, 10],
                         'n_estimators': [100, 200, 500]},
             verbose=10)

In [ ]:
rf = RandomForestClassifier(**rf_grid.best_params_)
lgb = lgb.LGBMClassifier(**lgb_grid.best_params_, n_jobs=-1)

In [ ]:
X_train = big_X_train
y_train = big_y_train

In [ ]:
from sklearn.ensemble import StackingClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import GradientBoostingClassifier


estimators = [
    ('rf', rf),
    ('svr', lgb),
    ('catboost', catboost)
]
model = StackingClassifier(
    estimators=estimators, final_estimator=GradientBoostingClassifier()
)

In [ ]:
model.fit(X_train, y_train)

0:	learn: 0.6408211	total: 34.6ms	remaining: 17.3s
1:	learn: 0.5946836	total: 59.4ms	remaining: 14.8s
2:	learn: 0.5539701	total: 90ms	remaining: 14.9s
3:	learn: 0.5170564	total: 123ms	remaining: 15.2s
4:	learn: 0.4838772	total: 150ms	remaining: 14.9s
5:	learn: 0.4554287	total: 178ms	remaining: 14.7s
6:	learn: 0.4303555	total: 206ms	remaining: 14.5s
7:	learn: 0.4065568	total: 236ms	remaining: 14.5s
8:	learn: 0.3886302	total: 263ms	remaining: 14.4s
9:	learn: 0.3685276	total: 290ms	remaining: 14.2s
10:	learn: 0.3512438	total: 316ms	remaining: 14s
11:	learn: 0.3384636	total: 342ms	remaining: 13.9s
12:	learn: 0.3246492	total: 368ms	remaining: 13.8s
13:	learn: 0.3119155	total: 393ms	remaining: 13.7s
14:	learn: 0.3001423	total: 419ms	remaining: 13.6s
15:	learn: 0.2903833	total: 448ms	remaining: 13.5s
16:	learn: 0.2810959	total: 473ms	remaining: 13.4s
17:	learn: 0.2717687	total: 499ms	remaining: 13.4s
18:	learn: 0.2630828	total: 523ms	remaining: 13.2s
19:	learn: 0.2559563	total: 548ms	remainin

StackingClassifier(estimators=[('rf', RandomForestClassifier(n_estimators=200)),
                               ('svr',
                                LGBMClassifier(learning_rate=0.05, max_depth=10,
                                               n_estimators=500)),
                               ('catboost',
                                <catboost.core.CatBoostClassifier object at 0x7f5ae3afdfd0>)],
                   final_estimator=GradientBoostingClassifier())

In [ ]:
!rm -rf public_data
!unzip public_data.zip > public_data_std_out
!rm public_data_std_out

In [ ]:
raw_data_to_predict = {}
for file in tqdm(glob('public_data/*')):
    key = int(os.path.basename(file).split('.')[0])
    with open(file, 'r', encoding='utf-8') as inp:
        raw_data_to_predict[key] = json.load(inp)
assert len(raw_data_to_predict.keys()) == 1000

  0%|          | 0/1000 [00:00<?, ?it/s]

In [ ]:
X_test = extract_features(raw_data_to_predict, final_features)
X_to_predict = preprocess_features(X_test)

0it [00:00, ?it/s]

defaultdict(<class 'list'>, {'dire_score': [55, 55, 47, 67, 45, 70, 66, 20, 59, 54, 38, 17, 27, 30, 36, 48, 51, 25, 39, 60, 29, 71, 62, 16, 47, 53, 11, 10, 60, 40, 35, 10, 36, 11, 30, 52, 67, 26, 23, 16, 39, 52, 73, 38, 55, 40, 32, 34, 52, 19, 14, 28, 45, 76, 55, 13, 13, 43, 63, 60, 22, 57, 53, 55, 52, 41, 41, 36, 29, 19, 40, 43, 11, 24, 47, 5, 48, 13, 19, 62, 45, 63, 51, 16, 61, 23, 27, 49, 51, 25, 36, 46, 50, 14, 19, 49, 15, 72, 33, 43, 58, 21, 56, 26, 27, 60, 38, 33, 40, 18, 24, 44, 33, 10, 40, 14, 10, 47, 24, 42, 59, 52, 39, 28, 22, 28, 49, 41, 15, 35, 33, 40, 33, 62, 50, 42, 48, 17, 48, 14, 23, 72, 70, 41, 48, 66, 59, 48, 39, 17, 53, 47, 28, 42, 49, 82, 78, 21, 20, 28, 19, 12, 49, 31, 36, 61, 60, 41, 46, 38, 55, 45, 46, 51, 27, 29, 72, 7, 22, 24, 31, 62, 14, 32, 26, 38, 35, 51, 20, 27, 6, 43, 12, 46, 50, 44, 7, 20, 42, 30, 34, 45, 56, 33, 41, 32, 40, 43, 39, 39, 66, 20, 54, 61, 22, 39, 65, 48, 62, 57, 58, 36, 22, 21, 32, 33, 39, 21, 72, 59, 17, 36, 57, 52, 19, 44, 29, 41, 60, 44, 

  0%|          | 0/132 [00:00<?, ?it/s]

In [ ]:
x_test_as_dict = X_to_predict.to_dict(orient='index')

In [ ]:
# получить X_standart_test
aug_x_test = make_augmented_dict(x_test_as_dict)
X_standart_test = pd.DataFrame.from_dict(aug_x_test, orient='index')
cat_cols = [col for col in X_standart_test.columns if X_standart_test[col].dtype not in ['int64', 'float64']]
X_standart_test.drop(columns=cat_cols, inplace=True)
X_standart_test = preprocess_features(X_standart_test)

  0%|          | 0/1000 [00:00<?, ?it/s]

  0%|          | 0/122 [00:00<?, ?it/s]

In [ ]:
X_test = X_standart_test

In [ ]:
probs = model.predict_proba(X_test)

In [ ]:
probs_for_game = probs[0::2][:, 1]
probs_for_reversed_game = probs[1::2][:, 0]
probs = (probs_for_game + probs_for_reversed_game) / 2

In [ ]:
problem_indexes = np.where(np.abs(probs - 0.5) <= 0.1)[0]
len(problem_indexes)

6

In [ ]:
from numpy.random import default_rng

rng = default_rng()

In [ ]:
import itertools

sep = '.'
sustainable_probs = []
for i in tqdm(problem_indexes):
    problem_index_augmentated = {}
    problem_game = x_test_as_dict[i]
    new_problem_game = problem_game.copy()
    indicies = [0, 1, 2, 3, 4]
    k = 0
    skip_indicies = rng.choice(120, size=60, replace=False)
    for perm_index, permutation in enumerate(itertools.permutations(indicies)):
        # if perm_index in skip_indicies:
        #     continue
        for key in problem_game.keys():
            key_parts = key.split(sep)
            if key_parts[0] == 'players' and int(key_parts[1]) < 5: # перестановка внутри первой команды
                new_key = key_parts[0] + sep + str(permutation[int(key_parts[1])]) + sep + sep.join(key_parts[2:])
                if new_key not in problem_game:
                    new_problem_game[key] = None
                else:
                    new_problem_game[key] = problem_game[new_key]
        skip_indicies2 = rng.choice(120, size=60, replace=False)
        for perm2_index, permutation2 in enumerate(itertools.permutations(indicies)):
            # if perm2_index in skip_indicies2:
            #     continue
            new_problem_game2 = new_problem_game.copy()
            for key in problem_game.keys():
                key_parts = key.split(sep)
                if key_parts[0] == 'players' and int(key_parts[1]) >= 5: # перестановка внутри второй команды
                    new_key = key_parts[0] + sep + str(permutation[int(key_parts[1]) - 5] + 5) + sep + sep.join(key_parts[2:])
                    if new_key not in problem_game:
                        new_problem_game2[key] = None
                    else:
                        new_problem_game2[key] = problem_game[new_key]
            problem_index_augmentated[k] = new_problem_game2
            k += 1
    X_problem = pd.DataFrame.from_dict(problem_index_augmentated, orient='index')

    cat_cols = [col for col in X_problem.columns if X_problem[col].dtype not in ['int64', 'float64']]
    X_problem.drop(columns=cat_cols, inplace=True)
    X_problem = preprocess_features(X_problem)
    X_problem.reset_index(drop=True, inplace=True)

    preds_problem = model.predict_proba(X_problem)

    sustainable_probs.append(np.mean(preds_problem[:, 1]))

  0%|          | 0/6 [00:00<?, ?it/s]

  0%|          | 0/122 [00:00<?, ?it/s]

  0%|          | 0/122 [00:00<?, ?it/s]

  0%|          | 0/122 [00:00<?, ?it/s]

  0%|          | 0/122 [00:00<?, ?it/s]

  0%|          | 0/122 [00:00<?, ?it/s]

  0%|          | 0/122 [00:00<?, ?it/s]

In [ ]:
sustainable_probs

[0.27918688796789853,
 0.35191289151943883,
 0.5373068786889726,
 0.6815215907799319,
 0.6509321440965853,
 0.42239149629083167]

In [ ]:
for i, pr in enumerate(sustainable_probs):
    probs[problem_indexes[i]] = pr

In [ ]:
labels = (probs >= 0.5).astype(int)
labels[:10]

array([0, 0, 0, 0, 1, 1, 1, 1, 0, 1])

In [ ]:
answer = pd.DataFrame()
answer['name'] = raw_data_to_predict.keys()
answer['target'] = labels

In [ ]:
answer.to_csv('simple_answer.csv', sep='\t', index=False)

In [ ]:
answer

,name,target
0,10013,0
1,10020,0
2,10037,0
3,10059,0
4,10113,1
...,...,...
995,9808,0
996,9860,1
997,9862,1
998,9866,1
